# Database Cracking Demo


```
 Copyright (C) 2017  LSBD Adaptive Databases Group
 
Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```

## Reference:

Idreos et. al. - Database Cracking - ICDE 2007

In [2]:
def crack_in_two(col, low, high, med):
    """
    Cracks a column partition into two subparitions
    and return te position of the new partition point
    """
    i = low
    j = high - 1
    while i <= j:
        if col[i] < med:
            i += 1
        else:
            while col[j] >= med and j > i:
                j -= 1
            col[i], col[j] = col[j], col[i]
            i += 1
            j -= 1
    return j

In [17]:
def crack_in_three(col, low, high, c1, c2):
    """
    Cracks a column partition into three subparitions
    and return te position of the new partition points
    """
    i = low  # keeps track of the initial lower bound
    j = high - 1 # keeps track of the initial upper bound
    swaps = 0 # keeps track of the number of swaps performed
    
    while col[j] >= c2 and j > i:
        j -= 1
        
    # from this point k is the lower bound
    # i is used as a supporting variable
    k = j
    
    while col[k] > c1 and k > i:
        if col[k] >= c2:
            col[j], col[k] = col[k], col[j]
            swaps += 1
            j -= 1
        k -= 1
    
    while i < k:
        if col[i] < c1:
            i += 1
        else:
            col[i], col[k] = col[k], col[i]
            swaps += 1
            while col[k] > c1 and k > i:
                if col[k] >= c2:
                    col[j], col[k] = col[k], col[j]
                    swaps += 1
                    j -= 1
                k -= 1
                
    return ((k, j), swaps)

## CrackInThree algorithm

Partitions the column so that all values in the selected range end up
clustered together

**Example:**

In [25]:
import random

# Numbers from 0 to 29
column = list(range(30))
random.shuffle(column)
print(column)

# Crack and reurn the answer to the user
print(crack_in_three(column, 0, len(column), 8, 20))
print(column)

[0, 2, 16, 6, 12, 10, 18, 8, 25, 5, 22, 3, 1, 15, 14, 23, 24, 7, 11, 26, 9, 20, 29, 21, 4, 27, 17, 28, 19, 13]
((8, 19), 16)
[0, 2, 4, 6, 7, 1, 3, 5, 8, 9, 19, 18, 10, 15, 14, 17, 13, 12, 11, 16, 25, 22, 23, 24, 26, 20, 29, 21, 27, 28]


## Cracker SELECT

Each query looks for the lower and upper boundary in the cracker index
and then cracks the partitions in the middle, if they aren't already in the
smallest size.

In [ ]:
import bintrees

# base column data (hundred million values)
column = list(range(100000000))
random.shuffle(column)

# cracker column to be reorganized (copy of original column)
cracker_column = column[:]

# cracker index. keeps track of the positions of the cracks
cracker_index = bintrees.AVLTree()